In [1]:
import pandas as pd
import os

In [2]:
# Both the abilities file and the typerank are highly subjective.  They were created using a combination of ngrams 
# as well as my own personal knowledge of the game.  These should be modified as you see fit.
dfabilities = pd.read_csv('abilities',header=None,names=['ability','rank'])
typerank = {'Enchantment':9,'Artifact':8,'Planeswalker':7,'Creature':5,'Instant':7,'Sorcery':1,'Land':10}

In [3]:
atraxa = []
for subdir,dirs,files in os.walk('Decklists/atraxa-praetors-voice/'):
    for file in files:
        deck = pd.read_csv(os.path.join(subdir,file))
        for dfindex,dfrows in deck.iterrows():
            ability_sum = 0
            for abilindex,abilrows in dfabilities.iterrows():
                try:
                    if dfrows['oracle_text'].lower().find(abilrows['ability'].lower()) > -1:
                        ability_sum += abilrows['rank']
                except:
                    continue
            deck.at[dfindex,'ability_score'] = ability_sum
            
        dfclean = deck[['cmc','type_line','loyalty','power','toughness','ability_score']].copy()
        for index,rows in dfclean.iterrows():
            typeline = rows['type_line']
            typewords = typeline.split()
            flag = True
            for a in typewords:
                if a in typerank.keys():
                    dfclean.at[index,'type_line'] = typerank[a]
                    flag = False
            if flag:
                dfclean.at[index,'type_line'] = 0
        dfclean=dfclean.fillna(0)
        
        # Variables like * and X need to be converted to integers
        for index,rows in dfclean.iterrows():
            if rows['loyalty'] == '*' or rows['loyalty'] == 'X':
                dfclean.at[index,'loyalty']=0
            if rows['power'] == '*' or rows['power'] == 'X':
                dfclean.at[index,'power']=0
            if rows['toughness'] == '*' or rows['toughness'] == 'X':
                dfclean.at[index,'toughness']=0
        
        # Columns need to be converted to ints
        dfclean['power'] = dfclean['power'].astype(int)
        dfclean['toughness'] = dfclean['toughness'].astype(int)
        dfclean['loyalty'] = dfclean['loyalty'].astype(int)
        
        # Normalize values by dividing by max
        dfclean['ability_score'] = dfclean['ability_score']/dfclean.apply(max)['ability_score']
        dfclean['type_line'] = dfclean['type_line']/dfclean.apply(max)['type_line']
        dfclean['loyalty'] = dfclean['loyalty']/dfclean.apply(max)['loyalty']
        dfclean['power'] = dfclean['power']/dfclean.apply(max)['power']
        dfclean['toughness'] = dfclean['toughness']/dfclean.apply(max)['toughness']
        dfclean['cmc'] = 1/dfclean.apply(max)['cmc']
        dfclean=dfclean.fillna(0)
        
        dfclean['card_power']=dfclean.sum(axis=1)/dfclean.astype(bool).sum(axis=1)
        
        atraxa.append(dfclean['card_power'].sum())

In [4]:
sum(atraxa)/len(atraxa)

38.26348639890193

In [6]:
atraxa_deck = []
deck = pd.read_csv('Decklists/atraxa-praetors-voice/atraxa-praetors-voice-_8jvd3UZ9N5nKc8mczqkxw.csv')
for dfindex,dfrows in deck.iterrows():
    ability_sum = 0
    for abilindex,abilrows in dfabilities.iterrows():
        try:
            if dfrows['oracle_text'].lower().find(abilrows['ability'].lower()) > -1:
                ability_sum += abilrows['rank']
        except:
            continue
    deck.at[dfindex,'ability_score'] = ability_sum

dfclean = deck[['cmc','type_line','loyalty','power','toughness','ability_score']].copy()
for index,rows in dfclean.iterrows():
    typeline = rows['type_line']
    typewords = typeline.split()
    flag = True
    for a in typewords:
        if a in typerank.keys():
            dfclean.at[index,'type_line'] = typerank[a]
            flag = False
    if flag:
        dfclean.at[index,'type_line'] = 0
dfclean=dfclean.fillna(0)

# Variables like * and X need to be converted to integers
for index,rows in dfclean.iterrows():
    if rows['loyalty'] == '*' or rows['loyalty'] == 'X':
        dfclean.at[index,'loyalty']=0
    if rows['power'] == '*' or rows['power'] == 'X':
        dfclean.at[index,'power']=0
    if rows['toughness'] == '*' or rows['toughness'] == 'X':
        dfclean.at[index,'toughness']=0

# Columns need to be converted to ints
dfclean['power'] = dfclean['power'].astype(int)
dfclean['toughness'] = dfclean['toughness'].astype(int)
dfclean['loyalty'] = dfclean['loyalty'].astype(int)

# Normalize values by dividing by max
dfclean['ability_score'] = dfclean['ability_score']/dfclean.apply(max)['ability_score']
dfclean['type_line'] = dfclean['type_line']/dfclean.apply(max)['type_line']
dfclean['loyalty'] = dfclean['loyalty']/dfclean.apply(max)['loyalty']
dfclean['power'] = dfclean['power']/dfclean.apply(max)['power']
dfclean['toughness'] = dfclean['toughness']/dfclean.apply(max)['toughness']
dfclean['cmc'] = 1/dfclean.apply(max)['cmc']
dfclean=dfclean.fillna(0)

dfclean['card_power']=dfclean.sum(axis=1)/dfclean.astype(bool).sum(axis=1)

atraxa_deck.append(dfclean['card_power'].sum())

In [7]:
sum(atraxa_deck)/len(atraxa_deck)

34.587852564102555

In [12]:
print(f'Average Atraxa deck power: {round(sum(atraxa)/len(atraxa),2)}')
print(f'Selected deck power: {round(sum(atraxa_deck)/len(atraxa_deck),2)}')

Average Atraxa deck power: 38.26
Selected deck power: 34.59
